In [3]:
import carmapy.carmapy_c
import numpy as np
import pandas as pd
from itertools import combinations

In [4]:
index_vec = np.array([0, 1], dtype=np.uint32)

Sigma = np.array([[1, 0.5], [0.5, 1]], dtype=np.float64)
z = np.array([1, 2], dtype=np.float64)

tau = 0.5
p_S = 1.0
y_sigma = 1.0
# Call cython function
result = carmapy.carmapy_c.Normal_fixed_sigma_marginal(index_vec, Sigma, z, tau, p_S, y_sigma)
# Result should be the same of the equivalent function in carma_rcpp_test.ipynb
print(result)


ValueError: Buffer dtype mismatch, expected 'int64_t' but got 'unsigned int'

In [6]:
from itertools import combinations
def set_gamma_func(input_S=None, condition_index=None, p=None):
    def set_gamma_func_base(S):
        def add_function(y):
            results = [np.sort([x] + y) for x in S_sub]
            return np.array(results)

        #S_sub = [i for i in range(0, p) if i not in S]
        set_gamma = [[], [], []]

        if len(S) == 0:
            S_sub = [i for i in range(0, p)]
            set_gamma[1] = [S + [x] for x in S_sub]
        elif len(S) == 1:
            S_sub = [i for i in range(0, p) if i not in S]
            set_gamma[0] = [list(comb) for comb in combinations(S, len(S) - 1)]
            set_gamma[1] = [sorted([x] + S) for x in S_sub]
            set_gamma[2] = add_function(set_gamma[0][0]).reshape(1, -1)
        else:
            S_sub = [i for i in range(0, p) if i not in S]
            set_gamma[0] = [sorted(list(comb)) for comb in combinations(S, len(S) - 1)] if len(S) > 2 else [list(comb) for comb in combinations(S, len(S) - 1)]
            set_gamma[1] = [sorted([x] + S) for x in S_sub]
            set_gamma[2] = add_function(set_gamma[0][0])
            for i in range(1, len(set_gamma[0])):
                set_gamma[2] = np.vstack((set_gamma[2], add_function(set_gamma[0][i])))
        return set_gamma

    def set_gamma_func_conditional(input_S, condition_index):
        def add_function(y):
            results = [np.sort([x] + y) for x in S_sub]
            return np.array(results)

        S = [i for i in input_S if i != condition_index]
        S_sub = [i for i in range(0, p) if i not in input_S]
        set_gamma = [[], [], []]

        if len(S) == 0:
            set_gamma[1] = [S + [x] for x in S_sub]
        elif len(S) == 1:
            set_gamma[0] = [list(comb) for comb in combinations(S, len(S) - 1)]
            set_gamma[1] = [sorted([x] + S) for x in S_sub]
            set_gamma[2] = add_function(set_gamma[0][0]).reshape(1, -1)
        else:
            set_gamma[0] = [sorted(list(comb)) for comb in combinations(S, len(S) - 1)] if len(S) > 2 else [list(comb) for comb in combinations(S, len(S) - 1)]
            set_gamma[1] = [sorted([x] + S) for x in S_sub]
            set_gamma[2] = add_function(set_gamma[0][0])
            for i in range(1, len(set_gamma[0])):
                set_gamma[2] = np.vstack((set_gamma[2], add_function(set_gamma[0][i])))
        return set_gamma

    if condition_index is None:
        results = set_gamma_func_base(input_S)
    else:
        results = set_gamma_func_conditional(input_S, condition_index)

    return results

# Example usage
p = 481
input_S = []
set_gamma = set_gamma_func(input_S, p=p)
set_gamma[1]


[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [78],
 [79],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [86],
 [87],
 [88],
 [89],
 [90],
 [91],
 [92],
 [93],
 [94],
 [95],
 [96],
 [97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104],
 [105],
 [106],
 [107],
 [108],
 [109],
 [110],
 [111],
 [112],
 [113],
 [114],
 [115],
 [116],
 [117],
 [118],
 [119],
 [120],
 [121],
 [122],
 [123],
 [124],
 [125],
 [126],
 [127],
 [128],
 [129],
 [130],
 [131],
 [132],
 [133],
 [134],
 [135],
 [136],
 [137],
 [138]

In [20]:
Sigma = np.matrix(np.loadtxt("/Users/hn9/Documents/GitHub/carmapy/sigma.csv", delimiter=","))
z = np.loadtxt("/Users/hn9/Documents/GitHub/carmapy/z.csv", delimiter=",")
tau_sample = 0.04
p_S = 481
y_var = 1
marginal_likelihood = carmapy.carmapy_c.Normal_fixed_sigma_marginal
set_gamma_margin = []
i=1
set_gamma_margin.append(np.apply_along_axis(marginal_likelihood, 1, np.array(set_gamma[1], dtype=np.uint32), Sigma=np.array(Sigma, dtype=np.float64), z=np.array(z, dtype=np.float64), tau=tau_sample, p_S=p_S, y_sigma=y_var))

In [21]:
set_gamma_margin

[array([-783.17346715, -783.57201985, -783.01877981, -782.25610169,
        -783.55543895, -783.14576476, -782.20690532, -782.74109336,
        -779.60721461, -782.02092875, -783.26273358, -768.58490436,
        -780.54064095, -783.1584446 , -781.89474526, -783.21729693,
        -782.14623211, -751.08473038, -781.34791091, -783.10243681,
        -783.24293906, -783.04326616, -783.47560622, -783.3395578 ,
        -771.91649613, -783.32298662, -783.26126047, -783.49529432,
        -782.97059982, -768.02457734, -783.26856035, -783.16256787,
        -783.51570248, -783.29186139, -783.51484295, -781.03713245,
        -783.5126983 , -782.48488906, -774.76529432, -783.28450893,
        -783.54604218, -783.30711329, -783.4491804 , -783.47334638,
        -783.51754907, -768.57987717, -779.46772051, -779.01699936,
        -779.06312785, -781.77730877, -783.40700656, -783.36214845,
        -782.26625955, -783.17346715, -783.01107722, -783.14338311,
        -782.43640833, -766.18951892, -765.65993

In [16]:
print('Sigma:', np.array(Sigma, dtype=np.float64))
print('z:', np.array(z, dtype=np.float64))
print('marginal likelihood function:', marginal_likelihood)
print('set_gamma[1]:', np.array(set_gamma[i], dtype=np.uint32))
print('tau sample', tau_sample)
print('p_S', p)
print('y_var', y_var)

Sigma: [[ 1.         -0.00840416 -0.03730959 ...  0.00717082 -0.00823892
  -0.00605219]
 [-0.00840416  1.         -0.05671948 ...  0.02630741  0.00952905
   0.013497  ]
 [-0.03730959 -0.05671948  1.         ...  0.01142669  0.00481769
  -0.00121849]
 ...
 [ 0.00717082  0.02630741  0.01142669 ...  1.          0.55963913
   0.50764549]
 [-0.00823892  0.00952905  0.00481769 ...  0.55963913  1.
   0.90459742]
 [-0.00605219  0.013497   -0.00121849 ...  0.50764549  0.90459742
   1.        ]]
z: [9.10714286e-01 2.02702703e-02 1.07291667e+00 1.65454545e+00
 1.86813187e-01 9.41818182e-01 1.68518519e+00 1.31481481e+00
 2.87179487e+00 1.79629630e+00 8.02325581e-01 5.58333333e+00
 2.51111111e+00 9.27710843e-01 1.86792453e+00 8.59205776e-01
 1.72222222e+00 8.22033898e+00 2.15094340e+00 9.88505747e-01
 8.27586207e-01 1.04891304e+00 4.48275862e-01 6.95652174e-01
 4.92380952e+00 7.20000000e-01 8.04232804e-01 4.00000000e-01
 1.11864407e+00 5.68674699e+00 7.94736842e-01 9.23076923e-01
 3.42857143e-01 7.

In [10]:
p = 2799
input_S = []
set_gamma = set_gamma_func(input_S, p=p)
set_gamma[1]

Sigma = np.matrix(np.loadtxt("/Users/hn9/Documents/GitHub/carmapy/carma_sigma.csv", delimiter=","))
z = np.loadtxt("/Users/hn9/Documents/GitHub/carmapy/carma_z.csv", delimiter=",")
tau_sample = 0.04
p_S = 1
y_var = 1
marginal_likelihood = carmapy.carmapy_c.Normal_fixed_sigma_marginal
set_gamma_margin = []
i=1
set_gamma_margin.append(np.apply_along_axis(marginal_likelihood, 1, np.array(set_gamma[1], dtype=np.int64), Sigma=np.array(Sigma, dtype=np.float64), z=np.array(z, dtype=np.float64), tau=tau_sample, p_S=p_S, y_sigma=y_var))

In [11]:
set_gamma_margin

[array([-1.52062013, -1.48342198, -1.41569708, ..., -1.20770527,
        -1.17015757, -1.61643113])]

In [13]:
# Testing on entire locus in one calculation (output number doesn't seem to match at scale with R version)
sumstats = pd.read_csv('/Users/hn9/Documents/GitHub/carmapy/tests/APOE_locus_sumstats.txt.gz', sep='\t')
ld = pd.read_csv('/Users/hn9/Documents/GitHub/carmapy/tests/APOE_locus_ld.txt.gz', sep='\t')

z = np.array(sumstats['Z'].tolist(), dtype=np.float64)
Sigma = np.array(ld,dtype=np.float64)

S = list(range(1, len(z)+1))
working_S = np.concatenate([np.array(sublist).flatten() for sublist in S])
working_S = np.unique(working_S - 1).astype(np.uint32)

tau = 0.04
p_S = 1.0
y_sigma = 1.0
# Call cython function
result = carmapy.carmapy_c.Normal_fixed_sigma_marginal(working_S, Sigma, z, tau, p_S, y_sigma)
# Result should be the same of the equivalent function in carma_rcpp_test.ipynb
print(result)


570898.544022032


In [6]:
index_vec_input = np.array([0, 1], dtype=np.uint32).astype(np.int32)  # Adjust indices to one-based indexing
Sigma = np.array([[1.0, 0.5],
                  [0.5, 1.0]], dtype=np.float64)
z = np.array([1.0, 2.0], dtype=np.float64)
tau = 0.5
p_S = 2
y_sigma = 1.0

result = carmapy.carmapy_c.outlier_Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)

-0.5228256304407761


# Additional C++ function tests:

In [7]:
index_vec_input = np.array([0, 1, 2], dtype=np.int32)  # Adjust indices to zero-based indexing
Sigma = np.array([[1.0, 0.5, 0.3],
                  [0.5, 2.0, 0.7],
                  [0.3, 0.7, 3.0]], dtype=np.float64)
z = np.array([1.5, 2.0, 3.5], dtype=np.float64)
tau = 0.1
p_S = 3  # Ensure correct value for p_S (size of Sigma matrix)
y_sigma = 0.01

result = carmapy.carmapy_c.outlier_Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)


-26.793893408406408


In [8]:
index_vec_input = np.array([0, 1], dtype=np.uint32)  # Adjust indices to zero-based indexing
Sigma = np.array([[1, 0.5], [0.5, 1]], dtype=np.float64)
z = np.array([1, 2], dtype=np.float64)

tau = 0.5
p_S = 2
y_sigma = 1.0


result = carmapy.carmapy_c.ind_Normal_fixed_sigma_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)

0.3352792291600821


In [9]:
import numpy as np
import carmapy

index_vec_input = np.array([1, 2], dtype=np.int32)  # Change dtype to int32
Sigma = np.array([[1.0, 0.5], [0.5, 1.0]], dtype=np.float64)
z = np.array([1.0, 2.0], dtype=np.float64)
tau = 0.8
p_S = 2  # p_S should be an integer, not a float
y_sigma = 1.0

result = carmapy.carmapy_c.outlier_ind_Normal_marginal(index_vec_input, Sigma, z, tau, p_S, y_sigma)
print(result)


-0.7345036438287167
